In [ ]:
%matplotlib inline
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import time

Environment demo
---

In [ ]:
%run env.py
%run rl-helpers.py

# Create drones & environment
drones_names = ['A', 'B', 'C']
env = DeliveryDrones(n_drones=len(drones_names))
drones = {
    drone_name: RandomAgent(
        state_size=env.observation_space.n,
        action_size=env.action_space.n
    )
    for drone_name in drones_names
}
states = env.reset(drones.keys())

# Simulation loop
while True:
    # Render
    clear_output(wait=True)
    env.render()
    
    # Act
    actions = {
        drone_name: drone_agent.act(states[drone_name])
        for drone_name, drone_agent in drones.items()
    }
    
    # Print last rewards and next actions
    print('Last rewards:', None if 'rewards' not in locals() else rewards)
    print('Next actions:', {n: Action(i) for n, i in actions.items()})
    
    # Sleep and step
    time.sleep(10)
    next_states, rewards, done, _ = env.step(actions)

Graphics
---

In [ ]:
# TODO

# Drone, pickup, dropoff, collision ~ simple geometric shapes
# https://image.freepik.com/free-vector/simple-geometric-shapes-background_1168-371.jpg
# Animation: slightly growing/shrinking to simulate up/down movement with shade

# Desired output
# https://img.deszone.net/2018/05/simple-geometric-shapes-free-vector-pattern4.jpg
# https://as1.ftcdn.net/jpg/01/72/82/18/500_F_172821814_Oyl3cNYBcigDpeCzehbAQghLxJILrZA5.jpg

# Other ideas
# Drones leave a fading trace